# Explanation of this notebook

This notebook got three purposes:
1. Improve the estimated delivery date given to customers at the purchase moment
2. Predict wether a delivery is gonna be late
3. Use clustering (k-means) to sugest where and how many warehouses to construct

# <font color='blue'>Setup 1</font>: Load Libraries

In [73]:
import numpy as np
import pandas as pd
import sys, os

# <font color='blue'>Setup 2</font>: Load data

In [74]:
def load_table(tablename):
    """ Defining the current directory and path to where data are located. Joining path to
    data folder and table name in order to read the table into a Pandas DataFrame"""
    
    file_path = os.path.join ('../data/') 

    df = os.path.join(file_path, tablename) 
    df = pd.read_csv(df, sep=',', encoding="latin1") 
    
    return df

In [75]:
# Reading the olist_orders table
orders = load_table('olist_orders_dataset.csv')
order_items = load_table('olist_order_items_dataset.csv')
customers = load_table('olist_customers_dataset.csv')
customers.columns = ['customer_id','customer_unique_id','geolocation_zip_code_prefix',
                     'customer_city','customer_state']
sellers = load_table('olist_sellers_dataset.csv')
sellers.columns = ['seller_id','geolocation_zip_code_prefix',
                     'seller_city','seller_state']
geo = load_table('olist_geolocation_dataset.csv')

# <font color='blue'>Setup 3</font>: Merge datasets

In [80]:
# Merging geo location to sellers and customers dataset
customers_geo = customers.merge(geo, on = "geolocation_zip_code_prefix", how = "left")
customers_geo = customers_geo[['customer_id', 'geolocation_lat','geolocation_lng']]
customers_geo.columns = ['customer_id', 'customer_lat','customer_lon']
customers_geo = customers_geo.groupby('customer_id')['customer_lat','customer_lon'].agg('mean').reset_index()

sellers_geo = sellers.merge(geo, on = "geolocation_zip_code_prefix", how = "left")
sellers_geo = sellers_geo[['seller_id', 'geolocation_lat','geolocation_lng']]
sellers_geo.columns = ['seller_id', 'seller_lat','seller_lon']
sellers_geo = sellers_geo.groupby('seller_id')['seller_lat','seller_lon'].agg('mean').reset_index()

In [92]:
# Merging geo location to orders dataset
order_items_all = order_items.merge(sellers_geo, on='seller_id', how='left')

orders_all = orders.merge(customers_geo, on='customer_id', how='left')
orders_all = orders_all.merge(order_items_all, on='order_id',how='left')

orders_all = orders_all[['order_id','order_status', 'customer_id', 'seller_id', 'product_id', 
                         'order_purchase_timestamp','order_approved_at',
                         'order_delivered_carrier_date','shipping_limit_date',
                         'order_delivered_customer_date','order_estimated_delivery_date','order_item_id',
                         'price','freight_value','customer_lat','customer_lon','seller_lat', 'seller_lon']]

In [93]:
# Filter all orders that has been delivered
delivered_orders = orders_all[orders_all['order_status']=='delivered']
delivered_orders = delivered_orders.drop('order_status', axis=1)

In [94]:
delivered_orders

,order_id,customer_id,seller_id,product_id,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,shipping_limit_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,price,freight_value,customer_lat,customer_lon,seller_lat,seller_lon
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,3504c0cb71d7fa48d967e0e4c94d59d9,87285b34884572647811a353c7ac498a,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-06 11:07:15,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,29.99,8.72,-23.576983,-46.587161,-23.680729,-46.444238
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,289cdb325fb7e7f891c38608bf9e0962,595fac2a385ac33a80bd5114aec74eb8,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-07-30 03:24:27,2018-08-07 15:27:45,2018-08-13 00:00:00,1.0,118.70,22.76,-12.177924,-44.660711,-19.807681,-43.980427
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,4869f7a5dfa277a7dca6462dcf3b52b2,aa4383b373c6aca5d8797843e5594415,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-13 08:55:23,2018-08-17 18:06:29,2018-09-04 00:00:00,1.0,159.90,19.22,-16.745150,-48.514783,-21.363502,-48.229601
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,66922902710d126a0e7d26b0e3805106,d0b61bfb1de832b15ba9d266ca96e5b0,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-11-23 19:45:59,2017-12-02 00:28:42,2017-12-15 00:00:00,1.0,45.00,27.20,-5.774190,-35.271143,-19.837682,-43.924053
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,2c9e548be18521d1c43cde1c582c6de8,65266b2da20d04dbe00c5c2d3bb7859e,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-19 20:31:37,2018-02-16 18:17:02,2018-02-26 00:00:00,1.0,19.90,8.72,-23.676370,-46.514627,-23.543395,-46.262086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113420,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,1f9ab4708f3056ede07124aad39a2554,f1d4ce8c6dd66c47bbaa8c6781c2a923,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-12 13:10:37,2018-02-28 17:37:56,2018-03-02 00:00:00,1.0,174.90,20.10,-24.001500,-46.449864,-21.930548,-50.498348
113421,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,d50d79cb34e38265a8649c383dcffd48,b80910977a37536adeddd63663f916ad,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-05 15:04:16,2017-09-21 11:24:17,2017-09-27 00:00:00,1.0,205.99,65.02,-17.898358,-39.373630,-23.553642,-46.452661
113422,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,a1043bafd471dff536d0c462352beb48,d1c427060a0f73f6b889a5c7c61f2ac4,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-12 21:36:21,2018-01-25 23:32:54,2018-02-15 00:00:00,1.0,179.99,40.59,-22.562825,-42.694574,-20.940578,-45.827237
113423,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,a1043bafd471dff536d0c462352beb48,d1c427060a0f73f6b889a5c7c61f2ac4,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-12 21:36:21,2018-01-25 23:32:54,2018-02-15 00:00:00,2.0,179.99,40.59,-22.562825,-42.694574,-20.940578,-45.827237


In [110]:
#Changing these columns to datetimes
delivered_orders.order_purchase_timestamp = pd.to_datetime(delivered_orders.order_purchase_timestamp)
delivered_orders.order_approved_at = pd.to_datetime(delivered_orders.order_approved_at)
delivered_orders.shipping_limit_date = pd.to_datetime(delivered_orders.shipping_limit_date)
delivered_orders.order_delivered_carrier_date = pd.to_datetime(delivered_orders.order_delivered_carrier_date)
delivered_orders.order_estimated_delivery_date = pd.to_datetime(delivered_orders.order_estimated_delivery_date)
delivered_orders.order_delivered_customer_date = pd.to_datetime(delivered_orders.order_delivered_customer_date)

In [99]:
#Calculating difference between shipping_limit_date and order_delivered_carrier_date
delivered_orders['shipping_difference'] = delivered_orders.shipping_limit_date - delivered_orders.order_delivered_carrier_date
delivered_orders['shipping_difference'] = delivered_orders['shipping_difference'].dt.days

In [104]:
delivered_orders['shipping_difference'].describe()

count    110195.000000
mean          2.823177
std           5.831250
min        -117.000000
25%           1.000000
50%           3.000000
75%           5.000000
max        1046.000000
Name: shipping_difference, dtype: float64

In [102]:
delivered_orders[delivered_orders['shipping_difference']<0]

,order_id,customer_id,seller_id,product_id,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,shipping_limit_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,price,freight_value,customer_lat,customer_lon,seller_lat,seller_lon,shipping_difference
21,203096f03d82e0dffbc41ebc2e2bcfb7,d2b091571da224a1b36412c18bc3bbfe,633ecdf879b94b5337cca303328e4a25,5ac9d9e379c606e36a8094a6046f75dc,2017-09-18 14:31:30,2017-09-19 04:04:09,2017-10-06 17:50:03,2017-09-25 04:04:09,2017-10-09 22:23:46,2017-09-28,1.0,109.90,8.96,-23.572939,-46.651115,-23.676570,-46.669897,-12.0
30,91b2a010e1e45e6ba3d133fa997597be,cce89a605105b148387c52e286ac8335,7d13fca15225358621be4086e1eb0964,ba74c6b75d2ad7503175809688d5a03c,2018-05-02 11:45:38,2018-05-03 12:55:01,2018-05-10 16:16:00,2018-05-09 12:55:01,2018-05-16 20:56:24,2018-05-23,1.0,178.99,13.69,-23.697396,-46.530322,-21.172861,-47.823700,-2.0
32,f70a0aff17df5a6cdd9a7196128bd354,456dc10730fbdba34615447ea195d643,5dceca129747e92ff8ef7a997dc4f8ca,cafd558df4c3c9d1c338ba6930ea9a62,2017-08-10 11:58:33,2017-08-12 02:45:24,2017-08-17 15:35:07,2017-08-17 02:45:24,2017-08-18 14:28:02,2017-08-23,1.0,279.00,34.19,-23.606375,-46.737491,-22.757883,-47.417055,-1.0
43,6ea2f835b4556291ffdc53fa0b3b95e8,c7340080e394356141681bd4c9b8fe31,f5f46307a4d15880ca14fab4ad9dfc9b,be021417a6acb56b9b50d3fd2714baa8,2017-11-24 21:27:48,2017-11-25 00:21:09,2017-12-13 21:14:05,2017-11-30 00:21:09,2017-12-28 18:59:23,2017-12-21,1.0,339.00,17.12,-21.874288,-51.844970,-27.231416,-49.657477,-14.0
62,a685d016c8a26f71a0bb67821070e398,911e4c37f5cafe1604fe6767034bf1ae,391fc6631aebcf3004804e51b40bcf1e,ebd7c847c1e1cb69ec374ae0ebee1f4c,2017-03-13 18:14:36,2017-03-13 18:14:36,2017-03-22 14:03:09,2017-03-17 18:14:36,2017-04-06 13:37:16,2017-03-30,1.0,84.90,14.36,-22.863075,-47.149981,-21.757321,-48.829744,-5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113376,b3112ca67f3afd4e20cf2ee91fc4f804,6f83c71b6c044fb156d697d4130fe9b5,5b0cc932433fa5184b5b94bfe6bcc256,4f18ca9862f511ecba98258b2194d061,2018-08-02 22:46:54,2018-08-02 23:04:06,2018-08-15 17:42:00,2018-08-08 23:04:06,2018-08-21 00:03:26,2018-08-16,1.0,95.00,24.75,-23.662299,-46.421695,-21.757321,-48.829744,-7.0
113377,b3112ca67f3afd4e20cf2ee91fc4f804,6f83c71b6c044fb156d697d4130fe9b5,5b0cc932433fa5184b5b94bfe6bcc256,4f18ca9862f511ecba98258b2194d061,2018-08-02 22:46:54,2018-08-02 23:04:06,2018-08-15 17:42:00,2018-08-08 23:04:06,2018-08-21 00:03:26,2018-08-16,2.0,95.00,24.75,-23.662299,-46.421695,-21.757321,-48.829744,-7.0
113395,e8fd20068b9f7e6ec07068bb7537f781,609b9fb8cad4fe0c7b376f77c8ab76ad,218d46b86c1881d022bce9c68a7d4b15,0df37da38a30a713453b03053d60d3f7,2017-08-10 21:21:07,2017-08-10 21:35:26,2017-08-18 16:42:51,2017-08-17 21:35:26,2017-08-23 15:36:29,2017-08-31,1.0,356.00,18.12,-23.544377,-46.565376,-21.112713,-47.790810,-1.0
113396,e8fd20068b9f7e6ec07068bb7537f781,609b9fb8cad4fe0c7b376f77c8ab76ad,218d46b86c1881d022bce9c68a7d4b15,0df37da38a30a713453b03053d60d3f7,2017-08-10 21:21:07,2017-08-10 21:35:26,2017-08-18 16:42:51,2017-08-17 21:35:26,2017-08-23 15:36:29,2017-08-31,2.0,356.00,18.12,-23.544377,-46.565376,-21.112713,-47.790810,-1.0


In [111]:
#Calculating difference between shipping_limit_date and order_delivered_carrier_date
delivered_orders['delivery_difference'] = delivered_orders.order_estimated_delivery_date - delivered_orders.order_delivered_customer_date
delivered_orders['delivery_difference'] = delivered_orders['delivery_difference'].dt.days

In [113]:
delivered_orders['delivery_difference'].describe()

count    110189.000000
mean         11.029041
std          10.158194
min        -189.000000
25%           6.000000
50%          12.000000
75%          16.000000
max         146.000000
Name: delivery_difference, dtype: float64

In [112]:
delivered_orders[delivered_orders['delivery_difference']<0]

,order_id,customer_id,seller_id,product_id,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,shipping_limit_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,price,freight_value,customer_lat,customer_lon,seller_lat,seller_lon,shipping_difference,delivery_difference
21,203096f03d82e0dffbc41ebc2e2bcfb7,d2b091571da224a1b36412c18bc3bbfe,633ecdf879b94b5337cca303328e4a25,5ac9d9e379c606e36a8094a6046f75dc,2017-09-18 14:31:30,2017-09-19 04:04:09,2017-10-06 17:50:03,2017-09-25 04:04:09,2017-10-09 22:23:46,2017-09-28,1.0,109.90,8.96,-23.572939,-46.651115,-23.676570,-46.669897,-12.0,-12.0
26,fbf9ac61453ac646ce8ad9783d7d0af6,3a874b4d4c4b6543206ff5d89287f0c3,c0563dd588b775f2e37747ef6ad6c92c,7b717060aa783eb7f23a747a3a733dd7,2018-02-20 23:46:53,2018-02-22 02:30:46,2018-02-26 22:25:22,2018-02-28 02:30:44,2018-03-21 22:03:54,2018-03-12,1.0,109.90,15.53,-22.879477,-43.440264,-23.622288,-46.536901,1.0,-10.0
37,8563039e855156e48fccee4d611a3196,5f16605299d698660e0606f7eae2d2f9,955fee9216a65b617aa5c0531780ce60,bff2010b28e8fbcff5a9db9d3fea5ac4,2018-02-17 15:59:46,2018-02-17 16:15:34,2018-02-20 23:03:56,2018-02-22 15:15:34,2018-03-20 00:59:25,2018-03-20,1.0,78.00,28.95,-12.478669,-49.122905,-23.691013,-46.703810,1.0,-1.0
43,6ea2f835b4556291ffdc53fa0b3b95e8,c7340080e394356141681bd4c9b8fe31,f5f46307a4d15880ca14fab4ad9dfc9b,be021417a6acb56b9b50d3fd2714baa8,2017-11-24 21:27:48,2017-11-25 00:21:09,2017-12-13 21:14:05,2017-11-30 00:21:09,2017-12-28 18:59:23,2017-12-21,1.0,339.00,17.12,-21.874288,-51.844970,-27.231416,-49.657477,-14.0,-8.0
60,66e4624ae69e7dc89bd50222b59f581f,684fa6da5134b9e4dab731e00011712d,db4350fd57ae30082dec7acbaacc17f9,b37b72d5a56f887725c2862184b8cab8,2018-03-09 14:50:15,2018-03-09 15:40:39,2018-03-15 00:31:19,2018-03-15 15:30:45,2018-04-03 13:28:46,2018-04-02,1.0,22.99,22.85,-10.971622,-37.067341,-23.580395,-46.591326,0.0,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113377,b3112ca67f3afd4e20cf2ee91fc4f804,6f83c71b6c044fb156d697d4130fe9b5,5b0cc932433fa5184b5b94bfe6bcc256,4f18ca9862f511ecba98258b2194d061,2018-08-02 22:46:54,2018-08-02 23:04:06,2018-08-15 17:42:00,2018-08-08 23:04:06,2018-08-21 00:03:26,2018-08-16,2.0,95.00,24.75,-23.662299,-46.421695,-21.757321,-48.829744,-7.0,-6.0
113378,0fa1fab1d7c1211c824596ed5e111e3c,7f3bd6c94d2daf7b6462d1a894a775b4,620c87c171fb2a6dd6e8bb4dec959fc6,149de4383ccb1829ea75380fc5f0b5fa,2018-03-13 21:48:57,2018-03-13 22:40:28,2018-03-14 19:27:23,2018-03-19 22:30:45,2018-04-05 19:59:49,2018-04-02,1.0,249.90,19.63,-23.568188,-46.594754,-22.540797,-43.201235,5.0,-4.0
113405,38e9133ce29f6bbe35aed9c3863dce01,ad312389a098ceff46ce92c4595c06d0,fa40cc5b934574b62717c68f3d678b6d,014a8a503291921f7b004a5215bb3c36,2017-10-12 20:54:11,2017-10-14 03:28:24,2017-10-17 17:04:42,2017-10-19 04:28:24,2017-11-21 17:06:59,2017-10-31,1.0,36.90,11.85,-21.772226,-48.180222,-23.472815,-46.600729,1.0,-22.0
113407,d692ef54145c9cb3322ec2e5508aa3f4,82ddfcf9438b0cd1117b55ac33184df8,701938c450705b8ae65fc923b70f35c7,58efb9b638561ce132216a9a612513e2,2018-03-21 19:47:18,2018-03-21 20:05:26,2018-03-22 21:11:58,2018-03-27 20:05:26,2018-04-11 00:48:31,2018-04-09,1.0,109.97,36.86,-22.951602,-47.001996,-20.818959,-49.377346,4.0,-3.0


# <font color='blue'>Part 1</font>: Descriptive analysis

In [90]:
def count_missing_values(df):
    missing = df.isna()
    num_missing = pd.DataFrame(missing.sum(),columns=['Number'])
    num_missing['Percentage'] = round(num_missing / len(df),4)
    
    print(num_missing)

In [91]:
count_missing_values(delivered_orders)

                               Number  Percentage
order_id                            0      0.0000
customer_id                         0      0.0000
seller_id                           0      0.0000
product_id                          0      0.0000
order_purchase_timestamp            0      0.0000
order_approved_at                  15      0.0001
order_delivered_carrier_date        2      0.0000
order_delivered_customer_date       8      0.0001
order_estimated_delivery_date       0      0.0000
order_item_id                       0      0.0000
shipping_limit_date                 0      0.0000
price                               0      0.0000
freight_value                       0      0.0000
customer_lat                      288      0.0026
customer_lon                      288      0.0026
seller_lat                        249      0.0023
seller_lon                        249      0.0023


# <font color='blue'>Part 2</font>: Prediction - Estimated delivery time

# <font color='blue'>Part 3</font>: Prediction - Late delivery (yes or no)

# <font color='blue'>Part 4</font>: Clustering - Warehouse locations